## Operations Reserach Club
## Optimization with Google OR tools

Please visit the following website for the tutorials and documentation. 

https://developers.google.com/optimization/introduction/python

We will maximize the usage of the operations room of a hospital. It has multiple op-rooms and multiple departments that are doing the operations in those op-rooms.

In [1]:
from ortools.linear_solver import pywraplp

In [2]:
def create_data_model():
    """Create the data for the example."""
    data = {}

    w = [36, 36, 48, 42, 42, 36, 24, 30, 30, 42, 36, 36]
    v = [50, 35, 30, 15, 40, 30, 35, 45, 10, 20, 30, 25]
    d= [0,0,0,1,1,1,1,1,1,1,1,1]
    l=list(range(12))
    items=list(zip(l,w,v,d)) # item_no, weight, value, department
    

    data['items'] = items
    
    num_bins = 5
    data['bins'] = list(range(num_bins))
    data['bin_capacities'] = [100, 100, 80, 80, 50]
    

   
    data['deps'] = list(set(list((x[-1]) for x in items )))
    data['dep_capacities'] = [400, 600]
    return data

In [3]:
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver('SCIP')

In [4]:
data=create_data_model()
data
   

{'items': [(0, 36, 50, 0),
  (1, 36, 35, 0),
  (2, 48, 30, 0),
  (3, 42, 15, 1),
  (4, 42, 40, 1),
  (5, 36, 30, 1),
  (6, 24, 35, 1),
  (7, 30, 45, 1),
  (8, 30, 10, 1),
  (9, 42, 20, 1),
  (10, 36, 30, 1),
  (11, 36, 25, 1)],
 'bins': [0, 1, 2, 3, 4],
 'bin_capacities': [100, 100, 80, 80, 50],
 'deps': [0, 1],
 'dep_capacities': [400, 600]}

In [5]:
# Variables
# x[i, j] = 1 if item i is packed in bin j.
x = {}
for i in data['items']:
    for j in data['bins']:
        x[(i[3],i[0], j)] = solver.IntVar(0, 1, 'x_%i_%i_%i' % (i[3],i[0], j))

In [6]:
len(x)

60

In [7]:
# Constraints
# Each item can be in at most one bin.
for i in data['items']:
    solver.Add(sum(x[(i[3],i[0], j)] for j in data['bins']) <= 1) # department_no, item_no, bin_no
    

print('Number of constraints =', solver.NumConstraints())

Number of constraints = 12


In [8]:
# The amount packed in each bin cannot exceed its capacity.
for j in data['bins']:
    solver.Add(
        sum(x[(i[3],i[0], j)] * i[1]
            for i in data['items']) <= data['bin_capacities'][j])
    
print('Number of constraints =', solver.NumConstraints())

Number of constraints = 17


In [9]:
# The dapertment cannot exceed its capacity

#for k in data['deps']:
for k in data['deps']:
    total=0
    for i in data['items'] :
        for j in data['bins']:
            if i[3]==k:
                total+= x[(i[3],i[0], j)]*i[1]
    solver.Add(total <= data['dep_capacities'][k])

In [10]:
print('Number of constraints =', solver.NumConstraints())

Number of constraints = 19


In [11]:
# Objective
objective = solver.Objective()

for i in data['items']:
    for j in data['bins']:
        objective.SetCoefficient(x[(i[3],i[0], j)], i[2])
objective.SetMaximization()

In [12]:
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
    print('Total packed value:', objective.Value())
    total_weight = 0
    for j in data['bins']:
        bin_weight = 0
        bin_value = 0
        print('Bin ', j, '\n')
        for i in data['items']:
            if x[(i[3],i[0], j)].solution_value() > 0:

                print('Item', i[0],'-department:', i[3], '- weight:', i[1], ' value:', i[2])
                bin_weight += i[1]
                bin_value += i[2]
        print('Packed bin weight:', bin_weight)
        print('Packed bin value:', bin_value)
        print()
        total_weight += bin_weight
    print('Total packed weight:', total_weight)
else:
    print('The problem does not have an optimal solution.')

Total packed value: 350.0
Bin  0 

Item 1 -department: 0 - weight: 36  value: 35
Item 7 -department: 1 - weight: 30  value: 45
Item 8 -department: 1 - weight: 30  value: 10
Packed bin weight: 96
Packed bin value: 90

Bin  1 

Item 0 -department: 0 - weight: 36  value: 50
Item 6 -department: 1 - weight: 24  value: 35
Item 10 -department: 1 - weight: 36  value: 30
Packed bin weight: 96
Packed bin value: 115

Bin  2 

Item 9 -department: 1 - weight: 42  value: 20
Item 11 -department: 1 - weight: 36  value: 25
Packed bin weight: 78
Packed bin value: 45

Bin  3 

Item 4 -department: 1 - weight: 42  value: 40
Item 5 -department: 1 - weight: 36  value: 30
Packed bin weight: 78
Packed bin value: 70

Bin  4 

Item 2 -department: 0 - weight: 48  value: 30
Packed bin weight: 48
Packed bin value: 30

Total packed weight: 396


In [16]:
print('dep capacities', data['deps'])
print('.....')
for dep in data['deps']:
    total_weight=0
    #print()
    for op_room in data['bins']:
        for item,weight,value,department in data['items']:
            if dep==department and x[(department,item,op_room)].solution_value()>0:
    
                total_weight+=weight
    print("Department", dep,":total weight", total_weight,"\n")


dep capacities [0, 1]
.....
Department 0 :total weight 120 

Department 1 :total weight 276 



In [14]:
data

{'items': [(0, 36, 50, 0),
  (1, 36, 35, 0),
  (2, 48, 30, 0),
  (3, 42, 15, 1),
  (4, 42, 40, 1),
  (5, 36, 30, 1),
  (6, 24, 35, 1),
  (7, 30, 45, 1),
  (8, 30, 10, 1),
  (9, 42, 20, 1),
  (10, 36, 30, 1),
  (11, 36, 25, 1)],
 'bins': [0, 1, 2, 3, 4],
 'bin_capacities': [100, 100, 80, 80, 50],
 'deps': [0, 1],
 'dep_capacities': [400, 600]}